In [4]:
import asyncio
from typing import Any

from metagpt.actions import Action, UserRequirement
from metagpt.logs import logger
from metagpt.roles import Role
from metagpt.schema import Message
from metagpt.team import Team

In [5]:
class SpeakAloud(Action):
    """Action: 清楚地阐述自己的观点"""

    PROMPT_TEMPLATE: str = """
    ## 背景
    设想你是 {name}, 你在与 {opponent_name}进行辩论.
    ## 辩论历史记录
    前面的轮次:
    {context}
    ## 轮到你了
    现在轮到你了，
    你应该紧密回应对手的最新论点，
    阐述你的立场，捍卫你的论据，
    并攻击对手的论点，
    构思一个深思熟虑的反驳，你将争论：
    """
    name: str = "SpeakAloud"

    async def run(self, context: str, name: str, opponent_name: str):
        prompt = self.PROMPT_TEMPLATE.format(context=context, name=name, opponent_name=opponent_name)
        # logger.info(prompt)

        rsp = await self._aask(prompt)

        return rsp

In [6]:
class Debator(Role):
    name: str = ""
    profile: str = ""
    opponent_name: str = ""

    def __init__(self, **data: Any):
        super().__init__(**data)
        self.set_actions([SpeakAloud])
        self._watch([UserRequirement, SpeakAloud])

    async def _observe(self) -> int:
        await super()._observe()
        # accept messages sent (from opponent) to self, disregard own messages from the last round
        self.rc.news = [msg for msg in self.rc.news if msg.send_to == {self.name}]
        return len(self.rc.news)

    async def _act(self) -> Message:
        logger.info(f"{self._setting}: to do {self.rc.todo}({self.rc.todo.name})")
        todo = self.rc.todo  # An instance of SpeakAloud

        memories = self.get_memories()
        context = "\n".join(f"{msg.sent_from}: {msg.content}" for msg in memories)
        # print(context)

        rsp = await todo.run(context=context, name=self.name, opponent_name=self.opponent_name)

        msg = Message(
            content=rsp,
            role=self.profile,
            cause_by=type(todo),
            sent_from=self.name,
            send_to=self.opponent_name,
        )
        self.rc.memory.add(msg)

        return msg

In [7]:
async def debate(idea: str, investment: float = 3.0, n_round: int = 5):
    """开启辩论过程，充分论证观点"""
    Zheng = Debator(name="Zheng", profile="正方", opponent_name="Fan")
    Fan = Debator(name="Fan", profile="反方", opponent_name="Zheng")
    team = Team()
    team.hire([Zheng, Fan])
    team.invest(investment)
    team.run_project(idea, send_to="Zheng") 
    await team.run(n_round=n_round)


async def main(idea: str, investment: float = 3.0, n_round: int = 5):
    """
    :param idea: 辩论主题
    :param investment: 监控花费的token，不要超出
    :param n_round: 辩论的最大轮次
    :return:
    """
    if platform.system() == "Windows":
        asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
    await debate(idea, investment, n_round)


await main(idea="中小型企业应该训练自己的大模型")

NameError: name 'platform' is not defined